## Examples

In this notebook some examples are generated. Given a query, the model returns similar sentences

In [2]:
from collections import defaultdict, Counter
import sys
import os
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from pathlib import Path
from pprint import pprint
import random 

import torch
from torch import nn, Tensor
from torch.utils import data as datautil
from torch.utils.data import Sampler, BatchSampler, DataLoader
import torch.nn.functional as F
from torch.optim import Optimizer
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
from dacite import from_dict

from matplotlib.pyplot import figure
from scipy import spatial

sys.path.append('../') 
from train import mmModule, multimodal_loss, Cfg
from utils import read_metadata_subset, find_paths, randomize_model
from dataloader import MMloader
from evaluate_topic import Evaluator

from omegaconf import OmegaConf
conf = OmegaConf.load("../config.yaml")

### Relocate some configuration files

In [3]:
# Rename configurations used, since they are one folder up.
conf.dataset_path = os.path.join("../", conf.dataset_path)
conf.sp_path = os.path.join("../", conf.sp_path)
conf.yamnet_query_embed_path = os.path.join("../", conf.yamnet_query_embed_path)
conf.yamnet_descr_embed_path = os.path.join("../", conf.yamnet_descr_embed_path)

conf.sent_topic_query_embed_dir = os.path.join("../", conf.sent_topic_query_embed_dir)
conf.sent_topic_descr_embed_dir = os.path.join("../", conf.sent_topic_descr_embed_dir)

metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')

qs =  topics_df['query'].tolist()
ds =  topics_df['description'].tolist()

[utils] metadata loaded  105360
[utils] topics_df_targets loaded for test set 9426
[utils] metadata_subset loaded  9426


In [3]:
model_weights_path = '../logs/25m-gru_2022-07-06_07-25-51/output/full_model_weights.pt'

In [4]:
# Reading the metadata.
metadata_testset, topics_df, topics_df_targets = read_metadata_subset(conf, traintest='test')


[utils] metadata loaded  105360
[utils] topics_df_targets loaded for test set 9426
[utils] metadata_subset loaded  9426


In [5]:
# Rename model paths. 
model_path = Path(model_weights_path).parents[1]
model_config_path = os.path.join(model_path, 'config.json')

# Opening JSON file
f = open(model_config_path)
model_config = json.load(f)
CFG = from_dict(data_class=Cfg, data=model_config)
CFG.device = "cuda" if torch.cuda.is_available() else "cpu"
CFG.sp_path = conf.sp_path

# Create dataloader of test set.
data_loader = MMloader(CFG)

# Load the model.
full_model = mmModule(CFG)
full_model.load_state_dict(torch.load(model_weights_path,  map_location=CFG.device)) 
full_model = full_model.to(CFG.device)     
full_model.eval()

# Calculate embeddings for test set.
evaluator = Evaluator(CFG, model_path, full_model, data_loader, False)
max_samples = evaluator.get_max_data()
print(max_samples)

[MMloader] init from directory  ../../data/yamnet/processed/ train
[MMloader] train dataset loaded, length:  4301835
[MMloader] init from directory  ../../data/yamnet/processed/ val
[MMloader] val dataset loaded, length:  252398
[MMloader] init from directory  ../../data/yamnet/processed/ test
[MMloader] test dataset loaded, length:  1530873
1530752


In [4]:

# For now we create a pool of 128*20000 samples
max_samples = 128*20000
evaluator.encode_testset(max_samples) 

Max samples reached


## Return top-k samples from a query

In [8]:

def get_topk_fromquery(evaluator, query, k=5):
    print("Input sentence: ", query)
    sent_txt_embed = evaluator.text_to_embed(query)
    
    # Create similarity matrix
    sim = (100.0 * sent_txt_embed @ evaluator.text_encoding.T)
    
    # return topK
    sim = sim.softmax(dim=-1)
    values, indices = sim.topk(k)
    topk_similar = [evaluator.all_sents[i] for i in indices]
    return topk_similar[0]


In [273]:
query = "Podcast about dogs."
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

Input sentence:  Podcast about dogs.
	 I'm going to talk about for dogs.
	 I'm obsessed with dogs.
	 It just says dogs.
	  Because I love dogs.
	 Thank God we both have dogs.


In [9]:

query = 'People are starving.'
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

Input sentence:  People are starving.
	 We are starving.
	 People are still starving.
	 You're not starving.
	 I was starving.
	 It's starving.


In [10]:
query = 'That is nice.'
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

Input sentence:  That is nice.
	 That is nice.
	  I was nice.
	 That was nice.
	 That was nice.
	 That was nice.


In [11]:
query = 'Dr. Gerber.'
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)


Input sentence:  Dr. Gerber.
	 Dr. Gerber.
	 He'd been a carpenter.
	 Dr. Pepper.
	 Dr. Bacher.
	  It buffer.com.


In [271]:

query = "Podcast about therapy."
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)

	 It was therapy.
	 I was just going to come a therapist.
	 Alternative therapy.
	 Welcome to Thrift therapy.
	 Physical therapy.


In [278]:

query = "Podcast about self-help."
topk_similar = get_topk_fromquery(evaluator, query, k=5)
for sent in topk_similar:
    print("\t", sent)


Input sentence:  Podcast about self-help books.
	 I find those self-help books.
	 We have enough good self-help stuff.
	 We were talking about self-doubt.
	 Let's go out self-love.
	 I find those self-help books.


### Return top-k from a query (cross modal)

In [23]:

def get_topk(evaluator, idx, mod='text-text', k=5):
    sent = evaluator.all_sents[idx]
    print("Input sentence: ", sent)
    
    # Retreive embeddings
    sent_txt_embed = evaluator.text_encoding[idx]
    sent_audio_embed = evaluator.audio_encoding[idx]
    
    # Create similarity matrices
    if mod=='text-text':
        sim = (100.0 * sent_txt_embed @ evaluator.text_encoding.T)
    elif mod=='text-audio':
        sim = (100.0 * sent_txt_embed @ evaluator.audio_encoding.T)
    elif mod=='audio-text':
        sim = (100.0 * sent_audio_embed @ evaluator.text_encoding.T)
    elif mod=='audio-audio':
        sim = (100.0 * sent_audio_embed @ evaluator.audio_encoding.T)
    
    # return topK
    sim = sim.softmax(dim=-1)
    values, indices = sim.topk(k)
    topk_similar = [evaluator.all_sents[i] for i in indices]
    return topk_similar


modalities = ['text-text', 'text-audio', 'audio-text', 'audio-audio']

In [63]:

idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent) 
        


Most similar for id 178989 and modality text-text
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 It's so strange.
prediction: 	 It was so strange.
prediction: 	 That's so strange.
prediction: 	 That's so strange.

Most similar for id 178989 and modality text-audio
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 That's so strange.
prediction: 	 He so strange.
prediction: 	 This is so strange.
prediction: 	 That's so strange.

Most similar for id 178989 and modality audio-text
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 Strange.
prediction: 	 It was so strange.
prediction: 	 It's so strange.
prediction: 	 It's so strange.

Most similar for id 178989 and modality audio-audio
Input sentence:  It's so strange.
prediction: 	 It's so strange.
prediction: 	 It's so strange.
prediction: 	 That was so strange.
prediction: 	 He goes.
prediction: 	 Yes, this is strange.


In [174]:

idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
        


Most similar for id 863328 and modality text-text
Input sentence:  What are you most thankful for this Thanksgiving?
prediction: 	 What are you most thankful for this Thanksgiving?
prediction: 	 What do you most thankful for for Thanksgiving?
prediction: 	 What are you plans for this wonderful Thanksgiving.
prediction: 	 I'm also grateful for sanity.
prediction: 	 Also intimacy.

Most similar for id 863328 and modality text-audio
Input sentence:  What are you most thankful for this Thanksgiving?
prediction: 	 Hey, what are you most thankful for this Thanksgiving.
prediction: 	 I was think of Thanksgiving.
prediction: 	 Tell them what you're thankful for.
prediction: 	 Happy Thanksgiving again.
prediction: 	 What are you most thankful for this Thanksgiving?

Most similar for id 863328 and modality audio-text
Input sentence:  What are you most thankful for this Thanksgiving?
prediction: 	 Let's go hold some heart space together.
prediction: 	 What are you most thankful for this Thanksgi

In [288]:

idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
        


Most similar for id 846526 and modality text-text
Input sentence:  I have a weight-loss goal.
prediction: 	 I have a weight-loss goal.
prediction: 	 Those are my goals.
prediction: 	 It makes or sport.
prediction: 	 I lost a ton of weight.
prediction: 	 So that was the score score score.

Most similar for id 846526 and modality text-audio
Input sentence:  I have a weight-loss goal.
prediction: 	 That's weight loss.
prediction: 	 Yeah, what are your next goals?
prediction: 	 I lost a ton of weight.
prediction: 	 I have a weight-loss goal.
prediction: 	 Those are my goals

Most similar for id 846526 and modality audio-text
Input sentence:  I have a weight-loss goal.
prediction: 	 Yeah, what are your next goals?
prediction: 	 What are my personal goals? 
prediction: 	 Let's talk about weight.
prediction: 	 What about your muscle mass?
prediction: 	 I have a weight-loss goal

Most similar for id 846526 and modality audio-audio
Input sentence:  I have a weight-loss goal.
prediction: 	 I ha

In [253]:
idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
        


Most similar for id 178989 and modality text-text
Input sentence:  So I was depressed probably for about five years.
prediction: 	 So I was depressed probably for about five years.
prediction: 	 I was the one that was fucking depressed.
prediction: 	 My son has been drug-free now over five years.
prediction: 	 But I was desperate.
prediction: 	 stop struggling.

Most similar for id 178989 and modality text-audio
Input sentence:  So I was depressed probably for about five years.
prediction: 	 So I was depressed probably for about five years.
prediction: 	 So I practice for five years.
prediction: 	 They're probably psyched as hell.
prediction: 	 But I was desperate.
prediction: 	 I'm a very independent person.

Most similar for id 178989 and modality audio-text
Input sentence:  So I was depressed probably for about five years.
prediction: 	 So I was depressed probably for about five years.
prediction: 	 I'm like no I never said I was depressed.
prediction: 	 I had feelings of depressio

In [254]:
idx = random.randint(0, len(evaluator.all_sents))

for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)
       


Most similar for id 178989 and modality text-text
Input sentence:  I don't think I made out with her like at the club.
prediction: 	 I don't think I made out with her like at the club.
prediction: 	 They've been more than like amicable.
prediction: 	 I don't even know how like, I'm an adult
prediction: 	 Definitely my favorite couple.
prediction: 	 So yeah, like half a kiss

Most similar for id 178989 and modality text-audio
Input sentence:  I don't think I made out with her like at the club.
prediction: 	 I don't think I made out with her like at the club.
prediction: 	 They've been more than like amicable.
prediction: 	 No way a couple.
prediction: 	 Give me a kiss.
prediction: 	 push Guinness down the toilet.

Most similar for id 178989 and modality audio-text
Input sentence:  I don't think I made out with her like at the club.
prediction: 	 I don't think I made out with her like at the club.
prediction: 	 No way a couple.
prediction: 	 They've been more than like amicable.
predict

In [257]:
idx = 470482
for mod in modalities:
    print("\nMost similar for id {} and modality {}".format(idx, mod))
    topk_similar = get_topk(evaluator, idx, mod=mod, k=5)
    for sent in topk_similar:
        print("prediction: \t", sent)


Most similar for id 470482 and modality text-text
Input sentence:  Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.

Most similar for id 470482 and modality text-audio
Input sentence:  Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.
prediction: 	 Anchor will distribute your podcast for you.

Most similar for id 470482 and modality audio-text
Input sentence:  Anchor will distribute your podcast for you.
prediction: 	 GE will distribute your podcast for you.
prediction: 	 